In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier

# Load the dataset
df = pd.read_csv(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\dataset\telecom_billing_dataset.csv")

# Add integer Invoice number for ordering
df['Invoice_Num_Int'] = df['Invoice_number'].str.replace("INV", "").astype(int)
df = df.sort_values(by='Invoice_Num_Int').reset_index(drop=True)

# Create Is_Duplicate flag
df['Is_Duplicate'] = (
    (df['Invoice_number'] == df['Invoice_number'].shift(1)) | 
    (df['Invoice_number'] == df['Invoice_number'].shift(-1))
).astype(int)

# Features and target
X = df.drop(columns=['Anomaly_type', 'Leakage'])
y = df[['Anomaly_type', 'Leakage']].copy()

# Handle missing values in targets
y['Anomaly_type'] = y['Anomaly_type'].fillna('Unknown').astype(str)
y['Leakage'] = y['Leakage'].fillna('Unknown').astype(str)

# Feature engineering for date columns
for col in ['Billing_date', 'Plan_start_date', 'Plan_end_date']:
    X[col] = pd.to_datetime(X[col], dayfirst=True)
    X[col + '_year'] = X[col].dt.year
    X[col + '_month'] = X[col].dt.month
    X[col + '_day'] = X[col].dt.day
X = X.drop(columns=['Billing_date', 'Plan_start_date', 'Plan_end_date'])

# Identify categorical and numerical features
categorical_features = X.select_dtypes(include=['object', 'category']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Transformers
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Encode targets
le_anomaly = LabelEncoder()
le_leakage = LabelEncoder()
y['Anomaly_type'] = le_anomaly.fit_transform(y['Anomaly_type'])
y['Leakage'] = le_leakage.fit_transform(y['Leakage'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Model
base_model = XGBClassifier(objective='binary:logistic',
                           eval_metric='logloss',
                           use_label_encoder=False,
                           random_state=42)
model = MultiOutputClassifier(base_model)

# Full pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Train
pipeline.fit(X_train, y_train)

# --- Evaluation ---
y_pred = pipeline.predict(X_test)

# Convert to DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=['Anomaly_type', 'Leakage'])
y_test_df = pd.DataFrame(y_test.values, columns=['Anomaly_type', 'Leakage'])

# Inverse transform
y_pred_df['Anomaly_type'] = le_anomaly.inverse_transform(y_pred_df['Anomaly_type'])
y_test_df['Anomaly_type'] = le_anomaly.inverse_transform(y_test_df['Anomaly_type'])
y_pred_df['Leakage'] = le_leakage.inverse_transform(y_pred_df['Leakage'])
y_test_df['Leakage'] = le_leakage.inverse_transform(y_test_df['Leakage'])

print("--- Evaluation for 'Anomaly_type' ---")
print(classification_report(y_test_df['Anomaly_type'], y_pred_df['Anomaly_type']))
print(f"Accuracy for 'Anomaly_type': {accuracy_score(y_test_df['Anomaly_type'], y_pred_df['Anomaly_type']):.4f}")

print("\n--- Evaluation for 'Leakage' ---")
print(classification_report(y_test_df['Leakage'], y_pred_df['Leakage']))
print(f"Accuracy for 'Leakage': {accuracy_score(y_test_df['Leakage'], y_pred_df['Leakage']):.4f}")

# --- Save the pipeline and encoders ---
joblib.dump(pipeline, r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\saved_model\telecom_pipeline.pkl")
joblib.dump(le_anomaly, r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\saved_model\le_anomaly.pkl")
joblib.dump(le_leakage, r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\saved_model\le_leakage.pkl")

print("✅ Model and encoders saved successfully!")


--- Evaluation for 'Anomaly_type' ---
                   precision    recall  f1-score   support

Duplicate entries       1.00      1.00      1.00       125
   Excess payment       1.00      1.00      1.00       137
 Extra data usage       1.00      0.99      1.00       106
  Missing charges       1.00      0.99      1.00       153
       No anomaly       0.99      1.00      0.99      1198
    Under payment       1.00      1.00      1.00       174
   Usage mismatch       1.00      0.88      0.94       107

         accuracy                           0.99      2000
        macro avg       1.00      0.98      0.99      2000
     weighted avg       0.99      0.99      0.99      2000

Accuracy for 'Anomaly_type': 0.9925

--- Evaluation for 'Leakage' ---
              precision    recall  f1-score   support

          No       0.99      1.00      0.99      1198
         Yes       1.00      0.98      0.99       802

    accuracy                           0.99      2000
   macro avg       0.9

In [6]:
import pandas as pd
import joblib

# Load pipeline and encoders
pipeline = joblib.load(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\saved_model\telecom_pipeline.pkl")
le_anomaly = joblib.load(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\saved_model\le_anomaly.pkl")
le_leakage = joblib.load(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\saved_model\le_leakage.pkl")

# Load new data (WITHOUT target columns)
new_df = pd.read_csv(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\dataset\telecom_input.csv")

# Feature engineering for dates (same as training)
for col in ['Billing_date', 'Plan_start_date', 'Plan_end_date']:
    new_df[col] = pd.to_datetime(new_df[col], dayfirst=True)
    new_df[col + '_year'] = new_df[col].dt.year
    new_df[col + '_month'] = new_df[col].dt.month
    new_df[col + '_day'] = new_df[col].dt.day
if 'Plan_start_date' in df.columns and 'Plan_end_date' in df.columns:
        new_df['No_of_valid_days'] = (
            (new_df['Plan_end_date'] - new_df['Plan_start_date']).dt.days
        ).fillna(0).astype(int) + 1
else:
    new_df['No_of_valid_days'] = 0
new_df = new_df.drop(columns=['Billing_date', 'Plan_start_date', 'Plan_end_date'])
print(new_df.columns)
# Make predictions
y_pred = pipeline.predict(new_df)

# Convert predictions back to original labels
y_pred_df = pd.DataFrame(y_pred, columns=['Anomaly_type', 'Leakage'])
y_pred_df['Anomaly_type'] = le_anomaly.inverse_transform(y_pred_df['Anomaly_type'])
y_pred_df['Leakage'] = le_leakage.inverse_transform(y_pred_df['Leakage'])

# Combine with original input data
output_df = pd.concat([new_df, y_pred_df], axis=1)

# Save to CSV
output_df.to_csv(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\output_dataset\telecom_predictions.csv", index=False)
no_leakage_df = output_df[output_df["Leakage"] == "No"]
anomaly_df = output_df[output_df["Leakage"] == "Yes"]

# Save them as separate CSVs
no_leakage_df.to_csv(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\output_dataset\telecom_no_leakage_data.csv", index=False)
anomaly_df.to_csv(r"D:\OneDrive\Desktop\revenue\ai-rev-leak\model\Telecom\output_dataset\telecom_anomaly_data.csv", index=False)
print("✅ Predictions saved to 'telecom_predictions.csv'")


Index(['Invoice_number', 'Customer_id', 'Service_id', 'Agent_id',
       'Payment_status', 'Transaction_type', 'Mode_of_payment', 'Plan_name',
       'Plan_category', 'Plan_charge', 'Tax_amount', 'Actual_amount',
       'Billed_amount', 'Paid_amount', 'Balance_amount', 'Zone_area',
       'Data_bought', 'Data_used', 'Billing_cycle', 'Billing_date_year',
       'Billing_date_month', 'Billing_date_day', 'Plan_start_date_year',
       'Plan_start_date_month', 'Plan_start_date_day', 'Plan_end_date_year',
       'Plan_end_date_month', 'Plan_end_date_day', 'No_of_valid_days'],
      dtype='object')
✅ Predictions saved to 'telecom_predictions.csv'
